# Lesson 6 - Advanced Examples

This lesson presents some non-standard uses of the package. To begin we first import the module as follows (and NumPy for array creation, matplotlib for plotting)

In [ ]:
from boldswimsuite import BOLDgeometry, BOLDspins, BOLDsequence
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm # for progress bars

The topics covered in this lesson are the following:

1. Saving/loading discrete voxels and continuous voxels
2. Applying Multiple Sequences to a Single Simulation
3. Adding a Constant Magnetic Field Offset to the Spins at Runtime (Gradient Field Offset example)

## 1. Saving/loading discrete voxels and continuous voxels

All voxels can be saved to a file and later loaded to run simulations. Continuous space voxels are saved using the Pickle package so the file extension is `.pkl`. Discrete space voxels are saved using NumPy's `savez` function so the file extension is `.npz`. In both cases, the voxel's `save` method is used to save the voxel to file and the `load` class method is used to load the voxel from the file.

> Note: Discrete voxels can require a lot of memory, especially with a high subvoxel count (N). This means that the save file for the voxel will also require quite a bit of storage space. Continuous voxel are analytically defined so their save files are much more light weight.

This lesson shows how to save and load a `ContinuousVoxel3D`, but the 'examples' folder contains example scripts for each kind of voxel:
- `saveload_ContinuousVoxel3D.py`

- `saveload_ContinuousVoxel2D.py`

- `saveload_DiscreteVoxel3D.py`

- `saveload_DiscreteVoxel2D.py`

First we randomly create a 3D continuous voxel:

In [ ]:
random_continuous_voxel = BOLDgeometry.ContinuousVoxel3D.from_random(
    size=0.2,
    CBV=0.02,
    B0=3,
    labels=['vein', 'artery'],
    weights={
        'vein':1, 
        'artery':1
    },
    diameter_distributions={
        'vein': [0.002, 0.003, 0.004], 
        'artery': [0.003, 0.004, 0.005]
    },
    dchis={
        'vein': 3e-8,
        'artery': 4e-8
    },
    permeation_probabilities={
        'vein': 0, 
        'artery': 0
    },
    vessel_type='cylinder',
    allow_vessel_intersection=True,
    seed=0, #repeating with the same seed with provide the same result
    progressbar=True
)


Now we create a string with the file path that we want to save the voxel to, and use the `save` method to save the voxel to file.

In [ ]:
filepath = r'random_ContinuousVoxel3D.pkl' #.pkl extension as this is a continuous voxel

random_continuous_voxel.save(filepath)

To load the voxel, we use the `load` method of the specific voxel object:

In [ ]:
loaded_voxel = BOLDgeometry.ContinuousVoxel3D.load(filepath)

We can now visualize the voxel to show that it has loaded properly:

In [ ]:
loaded_voxel.show()

## 2. Applying Multiple Sequences to a Single Simulation

One can apply multiple sequences to the same simulation. This can be useful in cases where different sequences are desired for the same spins and geometry parameters. Rather than performing 2 separate simulations we can use the same simulation but with multiple sequences, which is significantly more efficient. This method only works for Monte Carlo simulations using `Spins`.

To show how to do this, we start by creating a continuous voxel and a `Spins` object.

In [ ]:
random_continuous_voxel = BOLDgeometry.ContinuousVoxel3D.from_random(
    size=0.2,
    CBV=0.02,
    B0=3,
    labels=['vein', 'artery'],
    weights={
        'vein':1, 
        'artery':1
    },
    diameter_distributions={
        'vein': [0.002, 0.003, 0.004], 
        'artery': [0.003, 0.004, 0.005]
    },
    dchis={
        'vein': 3e-8,
        'artery': 4e-8
    },
    permeation_probabilities={
        'vein': 0, 
        'artery': 0
    },
    vessel_type='cylinder',
    allow_vessel_intersection=True,
    seed=0, #repeating with the same seed with provide the same result
    progressbar=True
)

print(random_continuous_voxel)

In [ ]:
num_spins = 10_000
dt=0.2 #we will use a constant 0.2ms time step

spins = BOLDspins.Spins3D(
    ADC=0.001,
    num_spins=num_spins,
    geometry=random_continuous_voxel,
    dt=dt,
    IV=True,
    seed=0 #repeating with the same seed with provide the same result
)

Now we cannot use the `SpinSequence` object to handle the sequence. This is because its `step` and `walk` methods also advances the `Spins`. We instead need to manually advance the `Spins` and then use its data to advance each sequence. To do this we use the `Sequence` object, which was shown in lesson 4.

In [ ]:
sequence1 = BOLDsequence.Sequence(
    sample_shape=spins.num_spins,
    pulse_time_indices=[0, 25],         #[0ms       , 5ms]
    pulse_angles=[np.pi/2, np.pi],      #[90 degrees, 180 degrees]
    pulse_axes=['y', 'x']               #[y-axis    , x-axis]
)

sequence2 = BOLDsequence.Sequence(
    sample_shape=spins.num_spins,
    pulse_time_indices=[0, 75],         #[0ms       , 15ms]
    pulse_angles=[np.pi/2, np.pi],      #[90 degrees, 180 degrees]
    pulse_axes=['y', 'x']               #[y-axis    , x-axis]
)

We now manually loop through the time steps and apply both sequences. Notice that we manually step through the `Spins`, since the `Sequence` object does not do it for us (unlike the `SpinSequence` object).

In [ ]:
num_steps = 200 # 200*0.2ms = 40ms

# signals for sequence 1
eviv1 = np.zeros(num_steps)
ev1 = np.zeros(num_steps)
iv1 = np.zeros(num_steps)

# signals for sequence 2
eviv2 = np.zeros(num_steps)
ev2 = np.zeros(num_steps)
iv2 = np.zeros(num_steps)

for j in tqdm(range(num_steps)): # use just `for i in range(num_steps)` to remove the progress bar  
    
    #step through the spins object
    spins.step(dt=dt)

    #collect spins state
    phase, vessel_indices, dt = spins.get_phase_vessel_indices_dt()

    #important step! Spins are intravascular only where vessel_indices is non-zero.
    is_IV = vessel_indices > 0
    
    #step through sequence 1 and save signals
    sequence1.step(phase=phase, is_IV=is_IV)
    eviv1[j], ev1[j], iv1[j] = sequence1.get_signals(cplx=False)

    #step through sequence 2 and save signals
    sequence2.step(phase=phase, is_IV=is_IV)
    eviv2[j], ev2[j], iv2[j] = sequence2.get_signals(cplx=False)

We can now plot both signals.

In [ ]:
# array of the time range
time_range = np.arange(0, dt*num_steps, dt)

# creating a matplotlib figure
figure, (ax1,ax2,ax3) = plt.subplots(nrows=1, ncols=3, figsize=(15,5))

# plotting all three signals with some formatting
ax1.plot(time_range, eviv1)
ax1.plot(time_range, eviv2)
ax1.set_title('Total')
ax1.set_xlabel('Time (ms)')
ax1.set_ylabel('Signal')

ax2.plot(time_range, ev1)
ax2.plot(time_range, ev2)
ax2.set_title('EV')
ax2.set_xlabel('Time (ms)')

ax3.plot(time_range, iv1)
ax3.plot(time_range, iv2)
ax3.set_title('IV')
ax3.set_xlabel('Time (ms)')

ax1.legend(['Sequence 1', 'Sequence 2'])

figure.tight_layout()

Notice that we could also save the `Spins` state at each time step to arrays, and then apply whatever sequence we want to that data later on. This is more memory demanding, since we save the `Spins` state at all time steps (whereas normally we discard the state at every step and only keep the signals). However this can be a good option if we want to save the simulation to file and be able to apply any sequence later on.

To do this we start by creating a new `Spins` object to run the simulation on.

In [ ]:
num_spins = 10_000
dt=0.2 #we will use a constant 0.2ms time step

spins = BOLDspins.Spins3D(
    ADC=0.001,
    num_spins=num_spins,
    geometry=random_continuous_voxel,
    dt=dt,
    IV=True,
    seed=0 #repeating with the same seed with provide the same result
)

Now we step through the simulation, but this time rather than having arrays to save each signal to, we instead create arrays to save the `Spins` state to.

In [ ]:
num_steps = 200 # 200*0.2ms = 40ms

phase_array = np.empty((num_steps, num_spins))
is_IV_array = np.empty((num_steps, num_spins), dtype=bool) #make sure this is set to bool dtype, otherwise this will default to float which is incorrect!
dt_array = np.empty(num_steps)

for j in tqdm(range(num_steps)): # use just `for i in range(num_steps)` to remove the progress bar  
    
    #step through the spins object
    spins.step(dt=dt)

    #collect spins state
    phase, vessel_indices, dt = spins.get_phase_vessel_indices_dt()

    #important step! Spins are intravascular only where vessel_indices is non-zero.
    is_IV = vessel_indices > 0

    phase_array[j, :] = phase
    is_IV_array[j, :] = is_IV
    dt_array[j] = dt

At this point we could save these three arrays to file using a function like `np.savez` and later on load that data (with `np.load` for example) into a different script that would apply the sequence(s). Here we will pretend this has already been done and we will simply show how to apply the sequences once the simulation data has been loaded in. We create a new sequence to show this (we could also have multiple sequences).

Let's say we don't remember the number of simulations steps or the number of spins. However, we know that `phase_array` has shape `(num_steps, num_spins)`, so we will recover this information into variables. We will need this information during the creation of the `Sequence` object and to step through it.

In [ ]:
num_steps, num_spins = phase_array.shape

sequence = BOLDsequence.Sequence(
    sample_shape=num_spins,
    pulse_time_indices=[0, 50],         #[0ms       , 10ms]
    pulse_angles=[np.pi/2, np.pi],      #[90 degrees, 180 degrees]
    pulse_axes=['y', 'x']               #[y-axis    , x-axis]
)

We now step through the sequence, but instead of getting the `Spins` state from the object, we take it from the saved simulation data.

In [ ]:
# signal arrays
eviv = np.zeros(num_steps)
ev = np.zeros(num_steps)
iv = np.zeros(num_steps)

for j in tqdm(range(num_steps)): # use just `for i in range(num_steps)` to remove the progress bar  
    
    #get the phase and is_IV data from the saved arrays
    phase = phase_array[j]
    is_IV = is_IV_array[j]
    
    #step through sequence 1 and save signals
    sequence.step(phase=phase, is_IV=is_IV)
    eviv[j], ev[j], iv[j] = sequence.get_signals(cplx=False)

And we plot to check if the simulation worked as expected.

In [ ]:
# array of the time range
time_range = np.arange(0, dt*num_steps, dt)

# creating a matplotlib figure
figure, (ax1,ax2,ax3) = plt.subplots(nrows=1, ncols=3, figsize=(15,5))

# plotting all three signals with some formatting
ax1.plot(time_range, eviv)
ax1.set_title('Total')
ax1.set_xlabel('Time (ms)')
ax1.set_ylabel('Signal')

ax2.plot(time_range, ev)
ax2.set_title('EV')
ax2.set_xlabel('Time (ms)')

ax3.plot(time_range, iv)
ax3.set_title('IV')
ax3.set_xlabel('Time (ms)')

figure.tight_layout()

## 3. Adding a Constant Magnetic Field Offset to the Spins at Runtime (Gradient Field Offset example)

Although gradient offsets are not currently implemented for voxels, they can be added to the spins at simulation runtime.

We first set up our simulation with the voxel, spins and sequence. We will once again use the `Sequence` object which will allow us to "fix" the spins' offset. We initially omit the gradient.

In [ ]:
random_continuous_voxel = BOLDgeometry.ContinuousVoxel3D.from_random(
    size=0.2,
    CBV=0.02,
    B0=3,
    labels=['vein', 'artery'],
    weights={
        'vein':1, 
        'artery':1
    },
    diameter_distributions={
        'vein': [0.002, 0.003, 0.004], 
        'artery': [0.003, 0.004, 0.005]
    },
    dchis={
        'vein': 3e-8,
        'artery': 4e-8
    },
    permeation_probabilities={
        'vein': 0, 
        'artery': 0
    },
    vessel_type='cylinder',
    allow_vessel_intersection=True,
    seed=0, #repeating with the same seed with provide the same result
    progressbar=True
)

print(random_continuous_voxel)

num_spins = 10_000
dt=0.2 #we will use a constant 0.2ms time step

spins = BOLDspins.Spins3D(
    ADC=0.001,
    num_spins=num_spins,
    geometry=random_continuous_voxel,
    dt=dt,
    IV=True,
    seed=0 #repeating with the same seed with provide the same result
)

sequence = BOLDsequence.Sequence(
    sample_shape=num_spins,
    pulse_time_indices=[0, 50],                     #[0ms       , 10ms]
    pulse_angles=[np.pi/2, np.pi],                  #[90 degrees, 180 degrees]
    pulse_axes=[[np.pi/2, np.pi/2], [np.pi/2, 0]]   #[y-axis    , x-axis]
)

We now define our gradient, based on the positions of spins. We will make a gradient in the x-direction that will start at $0 T$ and ramps up at $5\cdot 10^{-7} T/mm$.

We create a function that will take the spin positions and return the phase offset from the gradient for each position. Note that this approach would also work to add any constant magnetic field offset that can be represented as a function of position $f(x,y,z)$. In our case the gradient is only dependent on the x-position.

For implementation, we recall the following:
- `positions` is a (N, d) array of values, where N is the number of positions and d is the number of dimensions.
- Voxels are centered around 0, so they extend from `-size/2` to `size/2` in all directions.
- Because the spins store the phase offset and not the magnetic field offset, we have to convert the gradient to a phase gradient. We can access the gyromagnetic ratio from the `BOLDconstants` module. 

In [ ]:
from BOLDswimsuite.BOLDconstants import GYROMAGNETIC_RATIO #Hz/T

#to convert from magnetic field offset to phase offset
phase_conversion_factor = 2 * np.pi * GYROMAGNETIC_RATIO * dt * 0.001 #our dt is in ms but our gyromagnetic ratio is in Hz, so we multiply by 0.001

gradient_slope_T_per_mm = 5e-7 #T/mm


def calculate_gradient_phase_offset(positions):
    #get just the x component of the spin positions
    x = positions[:, 0]

    #adding size/2 will make our position values range from 0 to size instead of -size/2 to size/2
    #this makes it easier to apply the gradient
    x_shifted = x + random_continuous_voxel.size/2

    #calculate the magnetic field offset gradient for all positions
    gradient_T = x_shifted*gradient_slope_T_per_mm

    #convert to phase
    gradient_phase = gradient_T*phase_conversion_factor

    return gradient_phase

Now that our gradient function is created, we can manually add this additional phase offset to the spin phase offset in the simulation loop: $\Delta\phi_{total}(x,y,z,t) = \Delta\phi_{voxel}(x,y,z,t) + \Delta\phi_{gradient}(x,y,z)$

In [ ]:
num_steps = 200 # 200*0.2ms = 40ms

# signals for sequence 1
eviv = np.zeros(num_steps)
ev = np.zeros(num_steps)
iv = np.zeros(num_steps)

for j in tqdm(range(num_steps)): # use just `for i in range(num_steps)` to remove the progress bar  
    
    #step through the spins object
    spins.step(dt=dt)

    #collect spins state
    phase, vessel_indices, dt = spins.get_phase_vessel_indices_dt()

    #calculate the gradient phase offset for the current spin positions
    gradient_phase_offset = calculate_gradient_phase_offset(spins.positions)

    #add the gradient phase offset to the phase offset
    phase_total = phase + gradient_phase_offset

    #important step! Spins are intravascular only where vessel_indices is non-zero.
    is_IV = vessel_indices > 0
    
    #step through sequence 1 and save signals
    sequence.step(phase=phase_total, is_IV=is_IV)
    eviv[j], ev[j], iv[j] = sequence.get_signals(cplx=False)

We now plot the collected signals.

In [ ]:
# array of the time range
time_range = np.arange(0, dt*num_steps, dt)

# creating a matplotlib figure
figure, (ax1,ax2,ax3) = plt.subplots(nrows=1, ncols=3, figsize=(15,5))

# plotting all three signals with some formatting
ax1.plot(time_range, eviv)
ax1.set_title('Total')
ax1.set_xlabel('Time (ms)')
ax1.set_ylabel('Signal')

ax2.plot(time_range, ev)
ax2.set_title('EV')
ax2.set_xlabel('Time (ms)')

ax3.plot(time_range, iv)
ax3.set_title('IV')
ax3.set_xlabel('Time (ms)')

figure.tight_layout()